# Test dataset evaluation

In [1]:
import matplotlib.pyplot as plt

from ticl.evaluation.baselines import tabular_baselines

import seaborn as sns
import numpy as np
import warnings
warnings.simplefilter("ignore", FutureWarning)  # openml deprecation of array return type
from ticl.datasets import load_openml_list, open_cc_valid_dids, open_cc_dids
from ticl.evaluation.baselines.tabular_baselines import knn_metric, catboost_metric, logistic_metric, xgb_metric, random_forest_metric, mlp_metric, hyperfast_metric, hyperfast_metric_tuning, resnet_metric, mothernet_init_metric
from ticl.evaluation.tabular_evaluation import evaluate, eval_on_datasets, transformer_metric
from ticl.evaluation import tabular_metrics
from ticl.prediction.tabpfn import TabPFNClassifier

# Datasets

In [2]:
cc_test_datasets_multiclass, cc_test_datasets_multiclass_df = load_openml_list(open_cc_dids, multiclass=True, shuffled=True, filter_for_nan=False, max_samples = 10000, num_feats=100, return_capped=True)

Number of datasets: 30


# Setting params

In [3]:
import os
eval_positions = [1000]
max_features = 100
n_samples = 2000
base_path = os.path.join('../')
overwrite = False
max_times = [1, 15, 30, 60, 60 * 5, 60 * 15, 60*60]
metric_used = tabular_metrics.auc_metric
task_type = 'multiclass'

In [4]:
from ticl.evaluation.baselines.distill_mlp import DistilledTabPFNMLP
from ticl.prediction.mothernet import MotherNetClassifier
from functools import partial

# Baseline Evaluation
This section runs baselines and saves results locally.

In [5]:
!mkdir -p {base_path}/results
!mkdir -p {base_path}/results/tabular/
!mkdir -p {base_path}/results/tabular/multiclass/

In [6]:
cc_test_datasets_multiclass_df['isNumeric'] = (cc_test_datasets_multiclass_df.NumberOfSymbolicFeatures == 1) & (cc_test_datasets_multiclass_df.NumberOfInstancesWithMissingValues == 0)

In [7]:
cc_test_datasets_multiclass_df['NumberOfInstances'] =  cc_test_datasets_multiclass_df['NumberOfInstances'].astype(int)
cc_test_datasets_multiclass_df['NumberOfFeatures'] =  cc_test_datasets_multiclass_df['NumberOfFeatures'].astype(int)
cc_test_datasets_multiclass_df['NumberOfClasses'] =  cc_test_datasets_multiclass_df['NumberOfClasses'].astype(int)

print(cc_test_datasets_multiclass_df[['did', 'name', 'NumberOfFeatures', 'NumberOfInstances', 'NumberOfClasses']].rename(columns={'NumberOfFeatures': "d", "NumberOfInstances":"n", "NumberOfClasses": "k"}).to_latex(index=False))

\begin{tabular}{rlrrr}
\toprule
did & name & d & n & k \\
\midrule
11 & balance-scale & 5 & 625 & 3 \\
14 & mfeat-fourier & 77 & 2000 & 10 \\
15 & breast-w & 10 & 699 & 2 \\
16 & mfeat-karhunen & 65 & 2000 & 10 \\
18 & mfeat-morphological & 7 & 2000 & 10 \\
22 & mfeat-zernike & 48 & 2000 & 10 \\
23 & cmc & 10 & 1473 & 3 \\
29 & credit-approval & 16 & 690 & 2 \\
31 & credit-g & 21 & 1000 & 2 \\
37 & diabetes & 9 & 768 & 2 \\
50 & tic-tac-toe & 10 & 958 & 2 \\
54 & vehicle & 19 & 846 & 4 \\
188 & eucalyptus & 20 & 736 & 5 \\
458 & analcatdata_authorship & 71 & 841 & 4 \\
469 & analcatdata_dmft & 5 & 797 & 6 \\
1049 & pc4 & 38 & 1458 & 2 \\
1050 & pc3 & 38 & 1563 & 2 \\
1063 & kc2 & 22 & 522 & 2 \\
1068 & pc1 & 22 & 1109 & 2 \\
1462 & banknote-authentication & 5 & 1372 & 2 \\
1464 & blood-transfusion-service-center & 5 & 748 & 2 \\
1480 & ilpd & 11 & 583 & 2 \\
1494 & qsar-biodeg & 42 & 1055 & 2 \\
1510 & wdbc & 31 & 569 & 2 \\
6332 & cylinder-bands & 40 & 540 & 2 \\
23381 & dresses-sales

overlap:
balance-scale
mfeat-fourier
mfeat-karhunen
mfeat-morphological
credit-g
tic-tac-toe
vehicle
analcatdata_authorship
analcatdata_dmft
pc3
pc1
blood-transfusion-service-center
ilpd
qsar-biodeg
MiceProtein
car
steel-plates-fault
climate-simulation-model-crashes

non-overlap:
breast-w (valid)
mfeat-zernike (valid)
mcm (valid)
eucalyptus (valid)
wdbc (valid)
cylinder-bands (valid)
dresses-sales (valid)


banknote-authentication (test)
credit-approval (test)
diabetes (test)
pc4 (test)
kc2 (test)

In [8]:
from ticl.evaluation.tabular_evaluation import eval_on_datasets
max_times = [1, 5, 15, 60, 5 * 60, 15 * 60, 60* 60]

clf_dict= {
    'knn': knn_metric,
    'rf_new_params': random_forest_metric,
    'xgb': xgb_metric,
    'logistic': logistic_metric,
    'mlp': mlp_metric}

results_baselines = [
    eval_on_datasets('multiclass', model, model_name, cc_test_datasets_multiclass, eval_positions=eval_positions, max_times=max_times,
                     metric_used=metric_used, split_numbers=[1, 2, 3, 4, 5],
                     n_samples=n_samples, base_path=base_path, fetch_only=True)
    for model_name, model in clf_dict.items()
]

evaluating knn on cpu


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 96 concurrent workers.
[Parallel(n_jobs=-1)]: Done 173 tasks      | elapsed:    8.1s
[Parallel(n_jobs=-1)]: Done 525 tasks      | elapsed:    8.7s
[Parallel(n_jobs=-1)]: Done 1050 out of 1050 | elapsed:   17.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 96 concurrent workers.


evaluating rf_new_params on cpu


[Parallel(n_jobs=-1)]: Done 175 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 859 out of 1050 | elapsed:    0.6s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done 1050 out of 1050 | elapsed:    0.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 96 concurrent workers.


evaluating xgb on cpu


[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 859 out of 1050 | elapsed:    0.5s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done 1050 out of 1050 | elapsed:    0.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 96 concurrent workers.


evaluating logistic on cpu


[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 720 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 1050 out of 1050 | elapsed:    1.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 96 concurrent workers.


evaluating mlp on cpu


[Parallel(n_jobs=-1)]: Done 175 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 1050 out of 1050 | elapsed:    0.6s finished


In [9]:
from ticl.evaluation.tabular_evaluation import eval_on_datasets

max_times = [60 * 60]
clf_dict= {
    'hyperfast_tuning_gpu': hyperfast_metric_tuning}
results_hyperfast = [
    eval_on_datasets('multiclass', model, model_name, cc_test_datasets_multiclass, eval_positions=eval_positions, max_times=max_times,
                     metric_used=metric_used, split_numbers=[1, 2, 3, 4, 5],
                     n_samples=n_samples, base_path=base_path, fetch_only=False, device='cuda:2')
    for model_name, model in clf_dict.items()
]

evaluating hyperfast_tuning_gpu on cuda:2


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 150/150 [00:01<00:00, 122.59it/s]


In [10]:
max_times = [60 * 60]
clf_dict= {
    'mothernet_gd_gpu4': mothernet_init_metric}

results_mothernet_gd = [
    eval_on_datasets('multiclass', model, model_name, cc_test_datasets_multiclass, eval_positions=eval_positions, max_times=max_times,
                     metric_used=metric_used, split_numbers=[1, 2, 3, 4, 5],
                     n_samples=n_samples, base_path=base_path, n_jobs=1, device="cuda:0", verbose=0)
    for model_name, model in clf_dict.items()
]

evaluating mothernet_gd_gpu4 on cuda:0


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 150/150 [00:01<00:00, 123.10it/s]


In [11]:
# MLP GPU
max_times = [60 * 60]
# these will all be evaluated on CPU because they are given as callables, which is a weird way to do it.
clf_dict= {
    'mlp_gpu2': mlp_metric}

results_mlp = [
    eval_on_datasets('multiclass', model, model_name, cc_test_datasets_multiclass, eval_positions=eval_positions, max_times=max_times,
                     metric_used=metric_used, split_numbers=[1, 2, 3, 4, 5],
                     n_samples=n_samples, base_path=base_path, n_jobs=1, device="cuda:0", verbose=0)
    for model_name, model in clf_dict.items()
]

evaluating mlp_gpu2 on cuda:0


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 150/150 [00:01<00:00, 115.93it/s]


In [12]:
max_times = [60 * 60]
# these will all be evaluated on CPU because they are given as callables, which is a weird way to do it.
clf_dict= {
    'resnet_gpu': resnet_metric}

results_resnet = [
    eval_on_datasets('multiclass', model, model_name, cc_test_datasets_multiclass, eval_positions=eval_positions, max_times=max_times,
                     metric_used=metric_used, split_numbers=[1, 2, 3, 4, 5],
                     n_samples=n_samples, base_path=base_path, n_jobs=1, device="cuda:0", verbose=0)
    for model_name, model in clf_dict.items()
]

evaluating resnet_gpu on cuda:0


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 150/150 [00:01<00:00, 115.21it/s]


In [13]:
from ticl.evaluation.tabular_evaluation import eval_on_datasets
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import BaggingClassifier
from ticl.prediction.mothernet import ShiftClassifier, EnsembleMeta, MotherNetClassifier, MotherNetInitMLPClassifier
from sklearn.impute import SimpleImputer
from ticl.prediction.mothernet_additive import MotherNetAdditiveClassifier

from interpret.glassbox import ExplainableBoostingClassifier


from hyperfast import HyperFastClassifier

from sklearn import set_config
set_config(skip_parameter_validation=True, assume_finite=True)


# transformers don't have max times
import warnings
max_times = [1]
#device = "cuda:1"
device = "cpu"

ebm_default = ExplainableBoostingClassifier()
ebm_bins = ExplainableBoostingClassifier(max_bins=64)
ebm_bins_main_effects = ExplainableBoostingClassifier(max_bins=64, interactions=0)


model_string = "tabpfn_nooptimizer_emsize_512_nlayers_12_steps_2048_bs_32ada_lr_0.0001_1_gpu_07_24_2023_01_43_33"
tabpfn_ours = TabPFNClassifier(device=device, model_string=model_string, epoch="1650", N_ensemble_configurations=3)
mn_Dclass_average_03_25_2024_17_14_32_epoch_2910_ohe_ensemble_8 = EnsembleMeta(MotherNetClassifier(path="../models_diff/mn_Dclass_average_03_25_2024_17_14_32_epoch_2910.cpkt", device=device), n_estimators=8, onehot=True)
mn_Dclass_average_03_25_2024_17_14_32_epoch_2910_ohe_ensemble_8_quantile = EnsembleMeta(MotherNetClassifier(path="../models_diff/mn_Dclass_average_03_25_2024_17_14_32_epoch_2910.cpkt", device=device, inference_device=device), n_estimators=8, onehot=True, power="quantile")
mn_Dclass_average_03_25_2024_17_14_32_epoch_3970_ohe_ensemble_8_quantile = EnsembleMeta(MotherNetClassifier(path="../models_diff/mn_Dclass_average_03_25_2024_17_14_32_epoch_3970.cpkt", device=device), n_estimators=8, onehot=True, power="quantile")


mn_Dclass_average_03_25_2024_17_14_32_epoch_2910_ohe_ensemble_8_cpu_timing = EnsembleMeta(MotherNetClassifier(path="../models_diff/mn_Dclass_average_03_25_2024_17_14_32_epoch_2910.cpkt", device="cpu"), n_estimators=8, onehot=True)

mn_Dclass_average_03_25_2024_17_14_32_epoch_2910_ohe_ensemble_3 = EnsembleMeta(MotherNetClassifier(path="../models_diff/mn_Dclass_average_03_25_2024_17_14_32_epoch_2910.cpkt", device=device), n_estimators=3, onehot=True)

baam_nsamples500_numfeatures10_04_07_2024_17_04_53_epoch_1780 = MotherNetAdditiveClassifier(path="../models_diff/baam_nsamples500_numfeatures10_04_07_2024_17_04_53_epoch_1780.cpkt", device=device)

mn_categoricalfeaturep09_numfeaturessamplerdouble_sample_05_09_2024_23_39_30_epoch_1270_ohe_quantile_8_gpu_timing = EnsembleMeta(MotherNetClassifier(path="../models_diff/mn_categoricalfeaturep0.9_numfeaturessamplerdouble_sample_05_09_2024_23_39_30_epoch_1270.cpkt", device=device, inference_device=device), n_estimators=8, onehot=True, power="quantile")
mn_categoricalfeaturep09_numfeaturessamplerdouble_sample_05_09_2024_23_39_30_epoch_1270_ohe_quantile_8_cpu_timing = EnsembleMeta(MotherNetClassifier(path="../models_diff/mn_categoricalfeaturep0.9_numfeaturessamplerdouble_sample_05_09_2024_23_39_30_epoch_1270.cpkt", device="cpu"), n_estimators=8, onehot=True, power="quantile")


mlp_distill = make_pipeline(StandardScaler(), DistilledTabPFNMLP(n_epochs=1000, device=device, hidden_size=128, n_layers=2, dropout_rate=.1, learning_rate=0.01, model_string=model_string, epoch=1650, N_ensemble_configurations=3))
mothernet_21_46_25_3940_ensemble3 = EnsembleMeta(MotherNetClassifier(path="../models_diff/mn_d2048_H4096_L2_W32_P512_1_gpu_warm_08_25_2023_21_46_25_epoch_3940_no_optimizer.pickle", device=device), n_estimators=3)
ebm_bins_main_effects = ExplainableBoostingClassifier(max_bins=64, interactions=0)
baam_nfeatures_20_no_ensemble_e1520 = MotherNetAdditiveClassifier(path="../models_diff/baam_H512_Dclass_average_e128_nsamples500_numfeatures20_padzerosFalse_03_14_2024_15_03_22_epoch_1520.cpkt", device=device)
mothernet_init_gd_no_learning = MotherNetInitMLPClassifier(path="../models_diff/mn_Dclass_average_03_25_2024_17_14_32_epoch_2910.cpkt", device=device, n_epochs=1, learning_rate=0.0)


clf_dict= {
    #'mothernet': partial(transformer_metric, classifier=mothernet_21_46_25_3940_ensemble3, onehot=True),
    'mlp_distill': mlp_distill,
    'tabpfn': transformer_metric,
    #'mothernet_init_gd_no_learning': mothernet_init_gd_no_learning,
    'mothernet_init_gd_no_learning_preprocess2': partial(transformer_metric, classifier=mothernet_init_gd_no_learning, onehot=True),
    #'tabpfn_ours': tabpfn_ours,
    "hyperfast_no_optimize_gpu": partial(hyperfast_metric, optimization=None),
    "hyperfast_defaults_gpu": hyperfast_metric,
    #'ebm_default': partial(transformer_metric, classifier=ebm_default),
    #'ebm_bins': partial(transformer_metric, classifier=ebm_bins),
    #'ebm_bins_main_effects': partial(transformer_metric, classifier=ebm_bins_main_effects),
    #'baam_nfeatures_20_no_ensemble_e1520': baam_nfeatures_20_no_ensemble_e1520,
    #'mn_Dclass_average_03_25_2024_17_14_32_epoch_2910_ohe_ensemble_8': mn_Dclass_average_03_25_2024_17_14_32_epoch_2910_ohe_ensemble_8,
    #'mn_Dclass_average_03_25_2024_17_14_32_epoch_2910_ohe_ensemble_3': mn_Dclass_average_03_25_2024_17_14_32_epoch_2910_ohe_ensemble_3,
    #'baam_nsamples500_numfeatures10_04_07_2024_17_04_53_epoch_1780': baam_nsamples500_numfeatures10_04_07_2024_17_04_53_epoch_1780,

     'mothernet_ohe_ensemble_8_quantile_gpu_timing': mn_Dclass_average_03_25_2024_17_14_32_epoch_2910_ohe_ensemble_8_quantile,
    #'mn_Dclass_average_03_25_2024_17_14_32_epoch_2910_ohe_ensemble_8_quantile': mn_Dclass_average_03_25_2024_17_14_32_epoch_2910_ohe_ensemble_8_quantile,
    'mothernet_ohe_ensemble_8_gpu_timing_sklearn_refactor': mn_Dclass_average_03_25_2024_17_14_32_epoch_2910_ohe_ensemble_8,

    #'mn_Dclass_average_03_25_2024_17_14_32_epoch_3970_ohe_ensemble_8_quantile': mn_Dclass_average_03_25_2024_17_14_32_epoch_3970_ohe_ensemble_8_quantile,
    
    'mn_Dclass_average_03_25_2024_17_14_32_epoch_2910_ohe_ensemble_8_cpu_timing': mn_Dclass_average_03_25_2024_17_14_32_epoch_2910_ohe_ensemble_8_cpu_timing,
        'mn_Dclass_average_03_25_2024_17_14_32_epoch_2910_ohe_ensemble_8_cpu_timing_sklearn_refactor2': mn_Dclass_average_03_25_2024_17_14_32_epoch_2910_ohe_ensemble_8_cpu_timing,

    'mn_categoricalfeaturep09_numfeaturessamplerdouble_sample_05_09_2024_23_39_30_epoch_1270_ohe_quantile_8_gpu_timing': mn_categoricalfeaturep09_numfeaturessamplerdouble_sample_05_09_2024_23_39_30_epoch_1270_ohe_quantile_8_gpu_timing,
    'mn_categoricalfeaturep09_numfeaturessamplerdouble_sample_05_09_2024_23_39_30_epoch_1270_ohe_quantile_8_gpu_timing_sklearn_refactor': mn_categoricalfeaturep09_numfeaturessamplerdouble_sample_05_09_2024_23_39_30_epoch_1270_ohe_quantile_8_gpu_timing,
    'mn_categoricalfeaturep09_numfeaturessamplerdouble_sample_05_09_2024_23_39_30_epoch_1270_ohe_quantile_8_gpu_timing_again': mn_categoricalfeaturep09_numfeaturessamplerdouble_sample_05_09_2024_23_39_30_epoch_1270_ohe_quantile_8_gpu_timing,

    
    'mn_categoricalfeaturep09_numfeaturessamplerdouble_sample_05_09_2024_23_39_30_epoch_1270_ohe_quantile_8_cpu_timing3': mn_categoricalfeaturep09_numfeaturessamplerdouble_sample_05_09_2024_23_39_30_epoch_1270_ohe_quantile_8_cpu_timing,


    }
results_transformers = [
    eval_on_datasets('multiclass', model, model_name, cc_test_datasets_multiclass, eval_positions=eval_positions, max_times=max_times,
                     metric_used=metric_used, split_numbers=[1, 2, 3, 4, 5],
                     n_samples=n_samples, base_path=base_path, overwrite=False, n_jobs=-1, device=device)
    for model_name, model in clf_dict.items()
]

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 96 concurrent workers.
[Parallel(n_jobs=-1)]: Done  35 out of 150 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done 111 out of 150 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 96 concurrent workers.


evaluating mlp_distill on cpu
evaluating tabpfn on cpu


[Parallel(n_jobs=-1)]: Done  35 out of 150 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done 111 out of 150 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 96 concurrent workers.
[Parallel(n_jobs=-1)]: Done  35 out of 150 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done 111 out of 150 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 96 concurrent workers.


evaluating mothernet_init_gd_no_learning_preprocess2 on cpu
evaluating hyperfast_no_optimize_gpu on cpu


[Parallel(n_jobs=-1)]: Done  35 out of 150 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done 111 out of 150 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 96 concurrent workers.
[Parallel(n_jobs=-1)]: Done  35 out of 150 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done 111 out of 150 | elapsed:    0.1s remaining:    0.0s


...

/anaconda/envs/tabpfn_testing_environment/lib/python3.11/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [13 19] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/anaconda/envs/tabpfn_testing_environment/lib/python3.11/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
/anaconda/envs/tabpfn_testing_environment/lib/python3.11/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [13 19] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/anaconda/envs/tabpfn_testing_environment/lib/python3.11/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
/anaconda/envs/tabpfn_testing_environment/lib/python3.11/site-packages/sklearn/feature_selection/_univar

evaluating mn_categoricalfeaturep09_numfeaturessamplerdouble_sample_05_09_2024_23_39_30_epoch_1270_ohe_quantile_8_gpu_timing on cpu
evaluating mn_categoricalfeaturep09_numfeaturessamplerdouble_sample_05_09_2024_23_39_30_epoch_1270_ohe_quantile_8_gpu_timing_sklearn_refactor on cpu


[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:    0.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 96 concurrent workers.
[Parallel(n_jobs=-1)]: Done  35 out of 150 | elapsed:    0.7s remaining:    2.2s
[Parallel(n_jobs=-1)]: Done 111 out of 150 | elapsed:    0.7s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:    3.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 96 concurrent workers.
[Parallel(n_jobs=-1)]: Done  35 out of 150 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done 111 out of 150 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 96 concurrent workers.


evaluating mn_categoricalfeaturep09_numfeaturessamplerdouble_sample_05_09_2024_23_39_30_epoch_1270_ohe_quantile_8_gpu_timing_again on cpu
evaluating mn_categoricalfeaturep09_numfeaturessamplerdouble_sample_05_09_2024_23_39_30_epoch_1270_ohe_quantile_8_cpu_timing3 on cpu


[Parallel(n_jobs=-1)]: Done  35 out of 150 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done 111 out of 150 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:    0.1s finished


In [19]:
import pandas as pd
flat_results = []
for per_dataset in results_baselines + results_transformers + results_hyperfast + results_mlp + results_resnet + results_mothernet_gd:
    for result in per_dataset:
        row = {}
        for key in ['dataset', 'model', 'mean_metric', 'split', 'max_time']:
            row[key] = result[key]
        best_configs_key, = [k for k in result.keys() if "best_configs" in k]
        if result[best_configs_key][0] is not None:
            row.update(result[best_configs_key][0])
        row['mean_metric'] = float(row["mean_metric"].numpy())
        flat_results.append(row)

results_df = pd.DataFrame(flat_results)

In [20]:
results_df

,dataset,model,mean_metric,split,max_time,best,fit_time,inference_time,num_trials
0,balance-scale,knn,0.898451,1,1,{'n_neighbors': 14},0.000790,0.031152,NaN
1,balance-scale,knn,0.848925,2,1,{'n_neighbors': 8},0.000795,0.026842,NaN
2,balance-scale,knn,0.852651,3,1,{'n_neighbors': 10},0.000851,0.027301,NaN
3,balance-scale,knn,0.885874,4,1,{'n_neighbors': 10},0.000786,0.028868,NaN
4,balance-scale,knn,0.895205,5,1,{'n_neighbors': 15},0.000829,0.027410,NaN
...,...,...,...,...,...,...,...,...,...
7795,climate-model-simulation-crashes,mothernet_gd_gpu4,0.926609,1,3600,"{'dropout_rate': 0.3, 'learning_rate': 0.00057...",0.051890,0.000955,903.0
7796,climate-model-simulation-crashes,mothernet_gd_gpu4,0.937867,2,3600,"{'dropout_rate': 0.1, 'learning_rate': 1.01205...",0.256001,0.000964,902.0
7797,climate-model-simulation-crashes,mothernet_gd_gpu4,0.935806,3,3600,"{'dropout_rate': 0.1, 'learning_rate': 7.49013...",0.267680,0.001275,937.0
7798,climate-model-simulation-crashes,mothernet_gd_gpu4,0.937669,4,3600,"{'dropout_rate': 0.1, 'learning_rate': 2.63955...",0.247117,0.000973,934.0


In [21]:
#import pickle
#with open("results_test.pickle", "wb") as f:
#    pickle.dump(results_baselines + results_transformers, f)

In [22]:
results_df['model'] = results_df.model.replace({'knn': "KNN", 'rf_new_params': 'RF', 'mlp': "MLP",'mlp_distill': 'MLP-Distill', 'xgb':'XGBoost', 'logistic': 'LogReg',  'mothernet': 'MotherNet', 'tabpfn': 'TabPFN (Hollmann)', 'tabpfn_ours': 'TabPFN (ours)'})

In [23]:
import datetime
filename = f"results/results_test_for_mothernet_paper_{datetime.date.today()}.csv"
results_df.to_csv(filename)
filename

'results/results_test_for_mothernet_paper_2024-05-21.csv'